# Kerakli kutubxonalar yuklanadi

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import tensorflow_datasets as tfds

TensorFlow version: 2.10.1


# Datasetni yuklash

In [6]:
dataset = tfds.load("stanford_online_products", split = "train")
batch_size = 32
def preprocess_data(example):
    image = example['image']
    image = tf.image.resize(image, [50, 50])
    image = tf.cast(image, tf.float32) / 255.0
    label = example['super_class_id']
    return image, label

dataset = dataset.map(preprocess_data)

dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Arxitektura

In [8]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(12)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

Uning loss va accuracy parametrlari

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# Train funksiya

In [11]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

# Test funksiya

In [12]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

# Train jarayoni

In [13]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in dataset:
    train_step(images, labels)


  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 2.1552798748016357, Accuracy: 29.22201156616211, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 2, Loss: 1.7155224084854126, Accuracy: 43.801109313964844, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 3, Loss: 1.4132572412490845, Accuracy: 54.41218566894531, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 4, Loss: 1.095177173614502, Accuracy: 64.9594497680664, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 5, Loss: 0.8072302937507629, Accuracy: 74.30437469482422, Test Loss: 0.0, Test Accuracy: 0.0
